In [ ]:
import requests
import json
from datetime import datetime, timedelta
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
import joblib

# Initialize global variables
data_buffer = []

# Fetch data from the API
def fetch_data_from_api():
    url = "https://api.upstox.com/v2/market-quote/ohlc"
    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data from the API:", response.status_code)
        return None

# Process data, update buffer, and train model
def process_data_and_train_model(json_data, model):
    if json_data is None:
        return

    ohlc_data = json_data['data']['NSE_EQ:NHPC']['ohlc']
    timestamp = json_data['data']['NSE_EQ:NHPC']['timestamp']

    # Extract OHLC data and timestamp
    open_price = ohlc_data['open']
    high_price = ohlc_data['high']
    low_price = ohlc_data['low']
    close_price = ohlc_data['close']

    # Append new data to buffer
    data_buffer.append((timestamp, open_price, high_price, low_price, close_price))

    # Remove old data from buffer (keep only last 60 minutes data)
    current_time = datetime.fromisoformat(timestamp.split('+')[0])
    cutoff_time = current_time - timedelta(minutes=60)
    data_buffer[:] = [(t, o, h, l, c) for t, o, h, l, c in data_buffer if t >= cutoff_time]

    # Prepare features and target
    X = np.array([[o, h, l] for _, o, h, l, _ in data_buffer])
    y = np.array([c for _, _, _, _, c in data_buffer])

    # Train the model
    model.fit(X, y)

    # Save the trained model
    joblib.dump(model, 'trained_model.pkl')

# Load or initialize the model
def load_or_initialize_model():
    try:
        # Load the model if it exists
        model = joblib.load('trained_model.pkl')
        print("Model loaded successfully.")
    except:
        # Initialize the model if it doesn't exist
        model = RandomForestRegressor(random_state=42)
        print("Model initialized.")

    return model

# Main loop to continuously fetch data, update buffer, train model, predict, and display result
def main():
    model = load_or_initialize_model()

    while True:
        # Fetch data from the API
        json_data = fetch_data_from_api()
        if json_data is None:
            continue

        # Process data, update buffer, and train model
        process_data_and_train_model(json_data, model)

        # Predict OHLC after 10 minutes
        # Here, you can modify the prediction logic based on your requirement

# Run the main function
if __name__ == "__main__":
    main()


With Graph

In [ ]:
import requests
import json
from datetime import datetime, timedelta
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
import joblib
import time
import matplotlib.pyplot as plt

# Initialize global variables
data_buffer = []
last_training_date = None

# Fetch data from the API
def fetch_data_from_api():
    url = "https://api.upstox.com/v2/market-quote/ohlc"
    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data from the API:", response.status_code)
        return None

# Process data, update buffer, and train model
def process_data_and_train_model(json_data, model):
    if json_data is None:
        return

    ohlc_data = json_data['data']['NSE_EQ:NHPC']['ohlc']
    timestamp = json_data['data']['NSE_EQ:NHPC']['timestamp']

    # Extract OHLC data and timestamp
    open_price = ohlc_data['open']
    high_price = ohlc_data['high']
    low_price = ohlc_data['low']
    close_price = ohlc_data['close']

    # Append new data to buffer
    data_buffer.append((timestamp, open_price, high_price, low_price, close_price))

    # Remove old data from buffer (keep only last 60 minutes data)
    current_time = datetime.fromisoformat(timestamp.split('+')[0])
    cutoff_time = current_time - timedelta(minutes=60)
    data_buffer[:] = [(t, o, h, l, c) for t, o, h, l, c in data_buffer if t >= cutoff_time]

    # Prepare features and target
    X = np.array([[o, h, l] for _, o, h, l, _ in data_buffer])
    y = np.array([c for _, _, _, _, c in data_buffer])

    # Train the model only if it's a new day
    global last_training_date
    if last_training_date is None or last_training_date.date() != current_time.date():
        model.fit(X, y)
        last_training_date = current_time
        print("Model trained on:", last_training_date)

        # Save the trained model
        joblib.dump(model, 'trained_model.pkl')
        print("Model saved.")

# Load or initialize the model
def load_or_initialize_model():
    try:
        # Load the model if it exists
        model = joblib.load('trained_model.pkl')
        print("Model loaded successfully.")
    except:
        # Initialize the model if it doesn't exist
        model = RandomForestRegressor(random_state=42)
        print("Model initialized.")

    return model

# Plot data and future prediction
def plot_data_and_prediction(data_buffer, model):
    timestamps = [t for t, _, _, _, _ in data_buffer]
    close_prices = [c for _, _, _, _, c in data_buffer]
    last_timestamp = timestamps[-1]

    # Predict OHLC after 10 minutes
    future_timestamp = datetime.fromisoformat(last_timestamp.split('+')[0]) + timedelta(minutes=10)
    future_price = model.predict(np.array([[data_buffer[-1][1], data_buffer[-1][2], data_buffer[-1][3]]]))[0]

    # Plot data
    plt.figure(figsize=(10, 6))
    plt.plot(timestamps, close_prices, label='Close Price', color='blue')
    plt.scatter(last_timestamp, data_buffer[-1][4], color='red', label='Last Close Price')
    plt.scatter(future_timestamp, future_price, color='green', label='Predicted Close Price after 10 minutes')
    plt.xlabel('Timestamp')
    plt.ylabel('Close Price')
    plt.title('Last 60 Minutes Data and Future Prediction')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Main loop to continuously fetch data, update buffer, train model, predict, and display result
def main():
    model = load_or_initialize_model()

    while True:
        # Fetch data from the API
        json_data = fetch_data_from_api()
        if json_data is None:
            time.sleep(60)  # Wait for 1 minute if fetching data fails
            continue

        # Process data, update buffer, and train model
        process_data_and_train_model(json_data, model)

        # Plot data and future prediction
        plot_data_and_prediction(data_buffer, model)

# Run the main function
if __name__ == "__main__":
    main()


With sample values

In [ ]:
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
import joblib
import time
import matplotlib.pyplot as plt

# Initialize global variables
data_buffer = []
last_training_date = None

# Mock function to generate random OHLC data
def generate_mock_data():
    timestamp = datetime.now()
    open_price = np.random.uniform(50, 100)
    high_price = open_price + np.random.uniform(0, 5)
    low_price = open_price - np.random.uniform(0, 5)
    close_price = np.random.uniform(low_price, high_price)
    return {
        'data': {
            'NSE_EQ:NHPC': {
                'ohlc': {
                    'open': open_price,
                    'high': high_price,
                    'low': low_price,
                    'close': close_price
                },
                'timestamp': timestamp.isoformat()
            }
        }
    }

# Fetch data from the API (or mock data)
def fetch_data_from_api():
    # For testing, return mock data
    return generate_mock_data()

# Process data, update buffer, and train model
def process_data_and_train_model(json_data, model):
    if json_data is None:
        return

    try:
        ohlc_data = json_data['data']['NSE_EQ:NHPC']['ohlc']
        timestamp = json_data['data']['NSE_EQ:NHPC']['timestamp']

        # Extract OHLC data and timestamp
        open_price = ohlc_data['open']
        high_price = ohlc_data['high']
        low_price = ohlc_data['low']
        close_price = ohlc_data['close']

        # Append new data to buffer
        data_buffer.append((timestamp, open_price, high_price, low_price, close_price))

        # Remove old data from buffer (keep only last 60 minutes data)
        current_time = datetime.fromisoformat(timestamp.split('+')[0])
        cutoff_time = current_time - timedelta(minutes=60)
        data_buffer[:] = [(t, o, h, l, c) for t, o, h, l, c in data_buffer if datetime.fromisoformat(t.split('+')[0]) >= cutoff_time]

        # Prepare features and target
        X = np.array([[o, h, l] for _, o, h, l, _ in data_buffer])
        y = np.array([c for _, _, _, _, c in data_buffer])

        # Train the model only if it's a new day
        global last_training_date
        if last_training_date is None or last_training_date.date() != current_time.date():
            model.fit(X, y)
            last_training_date = current_time
            print("Model trained on:", last_training_date)

            # Save the trained model
            joblib.dump(model, 'trained_model.pkl')
            print("Model saved.")
    except Exception as e:
        print(f"Error processing data: {e}")

# Load or initialize the model
def load_or_initialize_model():
    try:
        # Load the model if it exists
        model = joblib.load('trained_model.pkl')
        print("Model loaded successfully.")
    except Exception as e:
        # Initialize the model if it doesn't exist
        model = RandomForestRegressor(random_state=42)
        print("Model initialized. Error loading model:", e)

    return model

# Plot data and future prediction
def plot_data_and_prediction(data_buffer, model):
    try:
        timestamps = [t for t, _, _, _, _ in data_buffer]
        close_prices = [c for _, _, _, _, c in data_buffer]
        last_timestamp = timestamps[-1]

        # Predict OHLC after 10 minutes
        future_timestamp = datetime.fromisoformat(last_timestamp.split('+')[0]) + timedelta(minutes=10)
        future_price = model.predict(np.array([[data_buffer[-1][1], data_buffer[-1][2], data_buffer[-1][3]]]))[0]

        # Plot data
        plt.figure(figsize=(10, 6))
        plt.plot(timestamps, close_prices, label='Close Price', color='blue')
        plt.scatter(last_timestamp, data_buffer[-1][4], color='red', label='Last Close Price')
        plt.scatter(future_timestamp, future_price, color='green', label='Predicted Close Price after 10 minutes')
        plt.xlabel('Timestamp')
        plt.ylabel('Close Price')
        plt.title('Last 60 Minutes Data and Future Prediction')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Error plotting data: {e}")

# Main loop to continuously fetch data, update buffer, train model, predict, and display result
def main():
    model = load_or_initialize_model()

    while True:
        try:
            # Fetch data from the API
            json_data = fetch_data_from_api()
            if json_data is None:
                time.sleep(60)  # Wait for 1 minute if fetching data fails
                continue

            # Process data, update buffer, and train model
            process_data_and_train_model(json_data, model)

            # Plot data and future prediction
            plot_data_and_prediction(data_buffer, model)

            # Sleep for 1 minute before fetching new data
            time.sleep(10)
        except Exception as e:
            print(f"Error in main loop: {e}")
            time.sleep(60)  # Wait for 1 minute before retrying

# Run the main function
if __name__ == "__main__":
    main()


code without the graph

In [1]:
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
import joblib
import time

# Initialize global variables
data_buffer = []
last_training_date = None

# Mock function to generate random OHLC data
def generate_mock_data():
    timestamp = datetime.now()
    open_price = np.random.uniform(50, 100)
    high_price = open_price + np.random.uniform(0, 5)
    low_price = open_price - np.random.uniform(0, 5)
    close_price = np.random.uniform(low_price, high_price)
    return {
        'data': {
            'NSE_EQ:NHPC': {
                'ohlc': {
                    'open': open_price,
                    'high': high_price,
                    'low': low_price,
                    'close': close_price
                },
                'timestamp': timestamp.isoformat()
            }
        }
    }

# Fetch data from the API (or mock data)
def fetch_data_from_api():
    # For testing, return mock data
    return generate_mock_data()

# Process data, update buffer, and train model
def process_data_and_train_model(json_data, model):
    if json_data is None:
        return

    try:
        ohlc_data = json_data['data']['NSE_EQ:NHPC']['ohlc']
        timestamp = json_data['data']['NSE_EQ:NHPC']['timestamp']

        # Extract OHLC data and timestamp
        open_price = ohlc_data['open']
        high_price = ohlc_data['high']
        low_price = ohlc_data['low']
        close_price = ohlc_data['close']

        # Append new data to buffer
        data_buffer.append((timestamp, open_price, high_price, low_price, close_price))

        # Remove old data from buffer (keep only last 60 minutes data)
        current_time = datetime.fromisoformat(timestamp.split('+')[0])
        cutoff_time = current_time - timedelta(minutes=60)
        data_buffer[:] = [(t, o, h, l, c) for t, o, h, l, c in data_buffer if datetime.fromisoformat(t.split('+')[0]) >= cutoff_time]

        # Prepare features and target
        X = np.array([[o, h, l] for _, o, h, l, _ in data_buffer])
        y = np.array([c for _, _, _, _, c in data_buffer])

        # Train the model only if it's a new day
        global last_training_date
        if last_training_date is None or last_training_date.date() != current_time.date():
            model.fit(X, y)
            last_training_date = current_time
            print("Model trained on:", last_training_date)

            # Save the trained model
            joblib.dump(model, 'trained_model.pkl')
            print("Model saved.")
    except Exception as e:
        print(f"Error processing data: {e}")

# Load or initialize the model
def load_or_initialize_model():
    try:
        # Load the model if it exists
        model = joblib.load('trained_model.pkl')
        print("Model loaded successfully.")
    except Exception as e:
        # Initialize the model if it doesn't exist
        model = RandomForestRegressor(random_state=42)
        print("Model initialized. Error loading model:", e)

    return model

# Function to print prediction
def print_prediction(data_buffer, model):
    try:
        last_timestamp = data_buffer[-1][0]

        # Predict OHLC after 10 minutes
        future_timestamp = datetime.fromisoformat(last_timestamp.split('+')[0]) + timedelta(minutes=10)
        future_price = model.predict(np.array([[data_buffer[-1][1], data_buffer[-1][2], data_buffer[-1][3]]]))[0]

        print(f"Predicted Close Price after 10 minutes at {future_timestamp}: {future_price}")
    except Exception as e:
        print(f"Error predicting data: {e}")

# Main loop to continuously fetch data, update buffer, train model, predict, and display result
def main():
    model = load_or_initialize_model()

    while True:
        try:
            # Fetch data from the API
            json_data = fetch_data_from_api()
            if json_data is None:
                time.sleep(60)  # Wait for 1 minute if fetching data fails
                continue

            # Process data, update buffer, and train model
            process_data_and_train_model(json_data, model)

            # Print the prediction
            print_prediction(data_buffer, model)

            # Sleep for 1 minute before fetching new data
            time.sleep(5)
        except Exception as e:
            print(f"Error in main loop: {e}")
            time.slee
            p(10)  # Wait for 1 minute before retrying

# Run the main function
if __name__ == "__main__":
    main()


Model loaded successfully.
Model trained on: 2024-06-09 19:42:02.494529
Model saved.
Predicted Close Price after 10 minutes at 2024-06-09 19:52:02.494529: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:07.627264: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:12.640719: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:17.651374: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:22.665858: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:27.673059: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:32.681508: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:37.704584: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:42.723669: 53.55362498110119
Predicted Close Price after 10 minutes at 2024-06-09 19:52:47.739857: 53.55362498110119
Predicted Close Price after 10 minu

KeyboardInterrupt: 

In [7]:
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
import joblib
import time

# Initialize global variables
data_buffer = []
last_training_date = None

# Mock function to generate random OHLC data
def generate_mock_data():
    timestamp = datetime.now()
    open_price = np.random.uniform(50, 100)
    high_price = open_price + np.random.uniform(0, 5)
    low_price = open_price - np.random.uniform(0, 5)
    close_price = np.random.uniform(low_price, high_price)
    return {
        'data': {
            'NSE_EQ:NHPC': {
                'ohlc': {
                    'open': open_price,
                    'high': high_price,
                    'low': low_price,
                    'close': close_price
                },
                'timestamp': timestamp.isoformat()
            }
        }
    }

# Fetch data from the API (or mock data)
def fetch_data_from_api():
    # For testing, return mock data
    return generate_mock_data()

# Process data, update buffer, and train model
def process_data_and_train_model(json_data, model):
    if json_data is None:
        return

    try:
        ohlc_data = json_data['data']['NSE_EQ:NHPC']['ohlc']
        timestamp = json_data['data']['NSE_EQ:NHPC']['timestamp']

        # Extract OHLC data and timestamp
        open_price = ohlc_data['open']
        high_price = ohlc_data['high']
        low_price = ohlc_data['low']
        close_price = ohlc_data['close']

        # Append new data to buffer
        data_buffer.append((timestamp, open_price, high_price, low_price, close_price))

        # Remove old data from buffer (keep only last 60 minutes data)
        current_time = datetime.fromisoformat(timestamp.split('+')[0])
        cutoff_time = current_time - timedelta(minutes=60)
        data_buffer[:] = [(t, o, h, l, c) for t, o, h, l, c in data_buffer if datetime.fromisoformat(t.split('+')[0]) >= cutoff_time]

        # Prepare features and target
        X = np.array([[o, h, l] for _, o, h, l, _ in data_buffer])
        y = np.array([c for _, _, _, _, c in data_buffer])

        # Train the model only if it's a new day
        global last_training_date
        if last_training_date is None or last_training_date.date() != current_time.date():
            model.fit(X, y)
            last_training_date = current_time
            print("Model trained on:", last_training_date)

            # Save the trained model
            joblib.dump(model, 'trained_model.pkl')
            print("Model saved.")
    except Exception as e:
        print(f"Error processing data: {e}")

# Load or initialize the model
def load_or_initialize_model():
    try:
        # Load the model if it exists
        model = joblib.load('trained_model.pkl')
        print("Model loaded successfully.")
    except Exception as e:
        # Initialize the model if it doesn't exist
        model = RandomForestRegressor(random_state=42)
        print("Model initialized. Error loading model:", e)

    return model

# Function to print prediction
def print_prediction(data_buffer, model):
    try:
        last_timestamp = data_buffer[-1][0]
        last_close_price = data_buffer[-1][4]

        # Predict OHLC after 10 minutes
        future_timestamp = datetime.fromisoformat(last_timestamp.split('+')[0]) + timedelta(minutes=10)
        future_price = model.predict(np.array([[data_buffer[-1][1], data_buffer[-1][2], data_buffer[-1][3]]]))[0]

        # Determine market movement
        market_move = "up" if future_price > last_close_price else "down" if future_price < last_close_price else "neutral"

        print(f"Current Close  at {last_timestamp}: {last_close_price}")
        print(f" after 10 minutes at {future_timestamp}: {future_price}")
        print(f"expected to move {market_move}")
    except Exception as e:
        print(f"Error predicting data: {e}")

# Main loop to continuously fetch data, update buffer, train model, predict, and display result
def main():
    model = load_or_initialize_model()

    while True:
        try:
            # Fetch data from the API
            json_data = fetch_data_from_api()
            if json_data is None:
                time.sleep(60)  # Wait for 1 minute if fetching data fails
                continue

            # Process data, update buffer, and train model
            process_data_and_train_model(json_data, model)

            # Print the prediction
            print_prediction(data_buffer, model)

            # Sleep for 1 minute before fetching new data
            time.sleep(2)
        except Exception as e:
            print(f"Error in main loop: {e}")
            time.sleep(2)  # Wait for 1 minute before retrying

# Run the main function
if __name__ == "__main__":
    main()


Model loaded successfully.
Model trained on: 2024-06-09 19:59:22.895610
Model saved.
Current Close  at 2024-06-09T19:59:22.895610: 56.88145402956563
 after 10 minutes at 2024-06-09 20:09:22.895610: 56.88145402956556
expected to move down
Current Close  at 2024-06-09T19:59:25.015543: 99.47617700434009
 after 10 minutes at 2024-06-09 20:09:25.015543: 56.88145402956556
expected to move down
Current Close  at 2024-06-09T19:59:27.023464: 65.75341780932399
 after 10 minutes at 2024-06-09 20:09:27.023464: 56.88145402956556
expected to move down
Current Close  at 2024-06-09T19:59:29.033026: 89.80474693749514
 after 10 minutes at 2024-06-09 20:09:29.033026: 56.88145402956556
expected to move down
Current Close  at 2024-06-09T19:59:31.047492: 100.88445818907391
 after 10 minutes at 2024-06-09 20:09:31.047492: 56.88145402956556
expected to move down
Current Close  at 2024-06-09T19:59:33.061745: 70.47535986192774
 after 10 minutes at 2024-06-09 20:09:33.061745: 56.88145402956556
expected to move d

KeyboardInterrupt: 